# Augmenting paths using variations of Dijkstra Algorithms


 GENERATING GRAPH AND ASSIGNING SOURCE AND SINK TO IT

In [20]:
import random
import math
from queue import Queue
from heapq import heappush, heappop
import heapq

In [4]:
def generate_sink_source_graph(n, r, upperCap):
    # Initialize vertices with random coordinates
    V = [{'x': random.uniform(0, 1), 'y': random.uniform(0, 1)} for _ in range(n)]
    
    E = set()

    # Edges based on distance and randomly choose direction
    for i in range(n):
        for j in range(n):
            if i != j:
                distance = math.sqrt((V[i]['x'] - V[j]['x'])**2 + (V[i]['y'] - V[j]['y'])**2)
                if distance <= r:
                    if random.random() < 0.5:
                        if ((i, j) not in E) and ((j, i) not in E):
                            E.add((i, j))
                    else:
                        if ((j, i) not in E) and ((i, j) not in E):
                            E.add((j, i))

    
    capacities = {edge: random.randint(1, upperCap) for edge in E}

    source = random.randint(0, n-1)

    # Breadth-first search to find the longest acyclic path
    def bfs_longest_path(src):
        visited = [False] * n
        queue = Queue()
        queue.put((src, [src]))

        longest_path = []
        while not queue.empty():
            current_node, path = queue.get()
            visited[current_node] = True

            for neighbor in [edge[1] for edge in E if edge[0] == current_node]:
                if not visited[neighbor]:
                    new_path = path + [neighbor]
                    queue.put((neighbor, new_path))
                    if len(new_path) > len(longest_path):
                        longest_path = new_path

        return longest_path

    longest_path = bfs_longest_path(source)
    sink = longest_path[-1] if longest_path else None

    return V, E, capacities, source, sink, longest_path



In [5]:
def save_graph_to_csv(graph, filename, source, sink):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for edge in graph.edges:
            writer.writerow([edge.start, edge.end, edge.capacity])

def read_graph_from_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        graph = defaultdict(lambda: defaultdict(int))
        for row in reader:
            start, end, capacity = map(int, row)
            graph[start][end] = capacity
    return graph

BFS algorithm for Augmenting Paths


In [6]:
def bfs_for_augmenting_path(graph, capacities, flow, start, end):
    parent = [-1] * len(graph)  # Store the augmenting path
    visited = [False] * len(graph)
    queue = Queue()

    queue.put(start)
    visited[start] = True

    while not queue.empty():
        u = queue.get()

        for v in range(len(graph)):
            if not visited[v] and (capacities.get((u, v), 0) - flow[u][v] > 0):
                queue.put(v)
                visited[v] = True
                parent[v] = u

    return parent, visited[end]

Ford Fulkerson Algorithm 

In [8]:
def ford_fulkerson(graph, capacities, source, sink):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]  # Initialize flow matrix

    max_flow = 0
    while True:
        parent, found_augmenting_path = bfs_for_augmenting_path(graph, capacities, flow, source, sink)

        if not found_augmenting_path:
            break

        # Find minimum residual capacity along the augmenting path
        path_flow = float('inf')
        s = sink
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]

        # Update flow for edges and reverse edges
        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow

    return max_flow



Augmenting Paths using Shortest Augmenting Path Algorithm

In [11]:
def dijkstra_sap(graph, capacities, flow, source, sink):
    n = len(graph)
    dist = [float('inf')] * n
    dist[source] = 0
    parent = [-1] * n
    heap = [(0, source)]

    while heap:
        d, u = heappop(heap)

        if u == sink:
            break

        for v in range(n):
            # Check for edge in residual graph
            if capacities.get((u, v), 0) - flow[u][v] > 0:
                # If we can improve the distance to v, update it
                if dist[u] + 1 < dist[v]:
                    dist[v] = dist[u] + 1
                    parent[v] = u
                    heappush(heap, (dist[v], v))

    return parent, dist[sink] != float('inf')


def ford_fulkerson_sap(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = dijkstra_sap(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    total_edges = len(capacities)
    mpl = (total_path_length / paths) / longest_path_length if paths > 0 else 0
    return max_flow, paths, mean_length, total_edges, mpl

In [12]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 4
Number of Augmenting Paths: 4
Average Length of Augmenting Paths: 10.5
Mean Proportional Length (MPL): 0.9545454545454546
Total Number of Edges: 522


In [13]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 7
Number of Augmenting Paths: 7
Average Length of Augmenting Paths: 6.0
Mean Proportional Length (MPL): 0.75
Total Number of Edges: 2120


In [14]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 5
Number of Augmenting Paths: 5
Average Length of Augmenting Paths: 6.4
Mean Proportional Length (MPL): 0.8
Total Number of Edges: 987


In [15]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 25
Number of Augmenting Paths: 23
Average Length of Augmenting Paths: 4.6521739130434785
Mean Proportional Length (MPL): 0.7753623188405797
Total Number of Edges: 4392


In [16]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 26
Number of Augmenting Paths: 6
Average Length of Augmenting Paths: 12.166666666666666
Mean Proportional Length (MPL): 0.869047619047619
Total Number of Edges: 517


In [17]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 203
Number of Augmenting Paths: 25
Average Length of Augmenting Paths: 5.84
Mean Proportional Length (MPL): 0.9733333333333333
Total Number of Edges: 2170


In [18]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 181
Number of Augmenting Paths: 31
Average Length of Augmenting Paths: 6.354838709677419
Mean Proportional Length (MPL): 0.9078341013824884
Total Number of Edges: 1141


In [19]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths:", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 493
Number of Augmenting Paths: 57
Average Length of Augmenting Paths: 3.8596491228070176
Mean Proportional Length (MPL): 0.7719298245614035
Total Number of Edges: 4142


Augmenting Path using DFS like algorithm 

In [21]:
def dfs_like_augmenting_path(graph, capacities, flow, start, end):
    n = len(graph)
    dist = [float('inf')] * n
    dist[start] = 0
    parent = [-1] * n
    heap = [(0, start)]
    counter = -1  # Decreasing counter for DFS-like behavior

    while heap:
        _, u = heapq.heappop(heap)

        for v in range(n):
            if capacities.get((u, v), 0) - flow[u][v] > 0:
                if dist[v] == float('inf'):
                    dist[v] = counter
                    counter -= 1
                    parent[v] = u
                    heapq.heappush(heap, (dist[v], v))

    return parent, dist[end] != float('inf')

def ford_fulkerson_dfs_like(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = dfs_like_augmenting_path(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    mpl = mean_length / longest_path_length if longest_path_length > 0 else 0
    total_edges = len(capacities)
    return max_flow, paths, mean_length, mpl, total_edges

In [72]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 2
Number of Augmenting Paths: 2
Average Length of Augmenting Paths (ML): 17.5
Mean Proportional Length (MPL): 1.5909090909090908
Total Number of Edges: 573


In [73]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 6
Number of Augmenting Paths: 6
Average Length of Augmenting Paths (ML): 24.166666666666668
Mean Proportional Length (MPL): 2.416666666666667
Total Number of Edges: 1989


In [25]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 16
Number of Augmenting Paths: 16
Average Length of Augmenting Paths (ML): 14.875
Mean Proportional Length (MPL): 2.4791666666666665
Total Number of Edges: 996


In [74]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 4
Number of Augmenting Paths: 4
Average Length of Augmenting Paths (ML): 36.5
Mean Proportional Length (MPL): 3.65
Total Number of Edges: 2034


In [75]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 27
Number of Augmenting Paths: 10
Average Length of Augmenting Paths (ML): 24.9
Mean Proportional Length (MPL): 1.9153846153846152
Total Number of Edges: 481


In [76]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 274
Number of Augmenting Paths: 182
Average Length of Augmenting Paths (ML): 26.576923076923077
Mean Proportional Length (MPL): 3.3221153846153846
Total Number of Edges: 2022


In [77]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 84
Number of Augmenting Paths: 23
Average Length of Augmenting Paths (ML): 11.565217391304348
Mean Proportional Length (MPL): 1.6521739130434783
Total Number of Edges: 1058


In [78]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 297
Number of Augmenting Paths: 142
Average Length of Augmenting Paths (ML): 18.471830985915492
Mean Proportional Length (MPL): 2.6388329979879273
Total Number of Edges: 4260


Augmenting Path using Maximum Cpacity

In [31]:
def maxcap_augmenting_path(graph, capacities, flow, start, end):
    n = len(graph)
    max_capacity = [0] * n
    max_capacity[start] = float('inf')
    parent = [-1] * n
    heap = [(-max_capacity[start], start)]

    while heap:
        _, u = heapq.heappop(heap)

        for v in range(n):
            if capacities.get((u, v), 0) - flow[u][v] > 0:
                path_capacity = min(max_capacity[u], capacities.get((u, v), 0) - flow[u][v])
                if path_capacity > max_capacity[v]:
                    max_capacity[v] = path_capacity
                    parent[v] = u
                    heapq.heappush(heap, (-max_capacity[v], v))

    return parent, max_capacity[end] > 0

def ford_fulkerson_maxcap(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = maxcap_augmenting_path(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    mpl = mean_length / longest_path_length if longest_path_length > 0 else 0
    total_edges = len(capacities)
    return max_flow, paths, mean_length, mpl, total_edges


In [79]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 2
Number of Augmenting Paths: 2
Average Length of Augmenting Paths (ML): 14.5
Mean Proportional Length (MPL): 1.2083333333333333
Total Number of Edges: 603


In [80]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 10
Number of Augmenting Paths: 6
Average Length of Augmenting Paths (ML): 10.833333333333334
Mean Proportional Length (MPL): 1.5476190476190477
Total Number of Edges: 2132


In [81]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 7
Number of Augmenting Paths: 5
Average Length of Augmenting Paths (ML): 9.0
Mean Proportional Length (MPL): 1.5
Total Number of Edges: 1140


In [82]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 24
Number of Augmenting Paths: 15
Average Length of Augmenting Paths (ML): 10.133333333333333
Mean Proportional Length (MPL): 1.6888888888888889
Total Number of Edges: 4523


In [83]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 27
Number of Augmenting Paths: 2
Average Length of Augmenting Paths (ML): 10.0
Mean Proportional Length (MPL): 1.1111111111111112
Total Number of Edges: 559


In [84]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 78
Number of Augmenting Paths: 6
Average Length of Augmenting Paths (ML): 21.0
Mean Proportional Length (MPL): 2.3333333333333335
Total Number of Edges: 2098


In [85]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 62
Number of Augmenting Paths: 2
Average Length of Augmenting Paths (ML): 8.5
Mean Proportional Length (MPL): 1.4166666666666667
Total Number of Edges: 1117


In [86]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

Maximum Flow: 318
Number of Augmenting Paths: 18
Average Length of Augmenting Paths (ML): 11.444444444444445
Mean Proportional Length (MPL): 1.6349206349206349
Total Number of Edges: 4108


Augmenting Path using Random DFS like algorithm

In [63]:
def random_dfs(graph, capacities, flow, start, end):
    n = len(graph)
    visited = [False] * n
    parent = [-1] * n

    def dfs(u):
        visited[u] = True
        if u == end:
            return True
        neighbors = [v for v in range(n) if capacities.get((u, v), 0) - flow[u][v] > 0]
        random.shuffle(neighbors)  # Randomize the order of neighbors
        for v in neighbors:
            if not visited[v]:
                parent[v] = u
                if dfs(v):
                    return True
        return False

    path_found = dfs(start)
    return parent, path_found

def ford_fulkerson_random(graph, capacities, source, sink, longest_path_length):
    n = len(graph)
    flow = [[0 for _ in range(n)] for _ in range(n)]
    paths = 0
    total_path_length = 0
    max_flow = 0

    while True:
        parent, found = random_dfs(graph, capacities, flow, source, sink)

        if not found:
            break

        path_flow = float('inf')
        s = sink
        path_length = 0
        while s != source:
            path_flow = min(path_flow, capacities.get((parent[s], s), 0) - flow[parent[s]][s])
            s = parent[s]
            path_length += 1

        v = sink
        while v != source:
            u = parent[v]
            flow[u][v] += path_flow
            flow[v][u] -= path_flow
            v = parent[v]

        max_flow += path_flow
        paths += 1
        total_path_length += path_length

    mean_length = total_path_length / paths if paths > 0 else 0
    mpl = mean_length / longest_path_length if longest_path_length > 0 else 0
    total_edges = len(capacities)
    return max_flow, paths, mean_length, mpl, total_edges

In [87]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 3
Number of Augmenting Paths: 3
Average Length of Augmenting Paths (ML): 36.333333333333336
Mean Proportional Length (MPL): 3.303030303030303
Total Number of Edges: 508


In [88]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 5
Number of Augmenting Paths: 5
Average Length of Augmenting Paths (ML): 91.0
Mean Proportional Length (MPL): 9.1
Total Number of Edges: 2177


In [89]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 15
Number of Augmenting Paths: 15
Average Length of Augmenting Paths (ML): 61.13333333333333
Mean Proportional Length (MPL): 10.188888888888888
Total Number of Edges: 1029


In [90]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 2)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 5
Number of Augmenting Paths: 5
Average Length of Augmenting Paths (ML): 130.8
Mean Proportional Length (MPL): 21.8
Total Number of Edges: 3987


In [91]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 37
Number of Augmenting Paths: 32
Average Length of Augmenting Paths (ML): 45.34375
Mean Proportional Length (MPL): 4.122159090909091
Total Number of Edges: 544


In [92]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 92
Number of Augmenting Paths: 91
Average Length of Augmenting Paths (ML): 128.4065934065934
Mean Proportional Length (MPL): 12.84065934065934
Total Number of Edges: 2105


In [93]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(100, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 298
Number of Augmenting Paths: 293
Average Length of Augmenting Paths (ML): 64.76450511945393
Mean Proportional Length (MPL): 10.794084186575654
Total Number of Edges: 1008


In [94]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(200, 0.3, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)


Maximum Flow: 127
Number of Augmenting Paths: 125
Average Length of Augmenting Paths (ML): 147.208
Mean Proportional Length (MPL): 21.029714285714284
Total Number of Edges: 4283


Trying the algorithms for different values of n and r

In [98]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(50, 1, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
longest_path_length = len(longest_path)
print("SAP")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("DFS like")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Max Cap")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Random")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

SAP
Maximum Flow: 799
Number of Augmenting Paths: 59
Average Length of Augmenting Paths (ML): 2.5762711864406778
Mean Proportional Length (MPL): 0.6440677966101694
Total Number of Edges: 1199
DFS like
Maximum Flow: 799
Number of Augmenting Paths: 91
Average Length of Augmenting Paths (ML): 4.538461538461538
Mean Proportional Length (MPL): 1.1346153846153846
Total Number of Edges: 1199
Max Cap
Maximum Flow: 799
Number of Augmenting Paths: 35
Average Length of Augmenting Paths (ML): 4.057142857142857
Mean Proportional Length (MPL): 1.0142857142857142
Total Number of Edges: 1199
Random
Maximum Flow: 799
Number of Augmenting Paths: 511
Average Length of Augmenting Paths (ML): 32.234833659491194
Mean Proportional Length (MPL): 8.058708414872799
Total Number of Edges: 1199


In [96]:
V, E, capacities, source, sink, longest_path = generate_sink_source_graph(50, 2, 50)
longest_path_length = len(longest_path)
max_flow, paths, mean_length, total_edges, mpl = ford_fulkerson_sap(V, capacities, source, sink, longest_path_length)
longest_path_length = len(longest_path)
print("SAP")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_dfs_like(V, capacities, source, sink, longest_path_length)
print("DFS like")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_maxcap(V, capacities, source, sink, longest_path_length)
print("Max Cap")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)
max_flow, paths, mean_length, mpl, total_edges = ford_fulkerson_random(V, capacities, source, sink, longest_path_length)
print("Random")
print("Maximum Flow:", max_flow)
print("Number of Augmenting Paths:", paths)
print("Average Length of Augmenting Paths (ML):", mean_length)
print("Mean Proportional Length (MPL):", mpl)
print("Total Number of Edges:", total_edges)

SAP
Maximum Flow: 564
Number of Augmenting Paths: 51
Average Length of Augmenting Paths (ML): 2.843137254901961
Mean Proportional Length (MPL): 0.7107843137254902
Total Number of Edges: 1225
DFS like
Maximum Flow: 564
Number of Augmenting Paths: 65
Average Length of Augmenting Paths (ML): 4.538461538461538
Mean Proportional Length (MPL): 1.1346153846153846
Total Number of Edges: 1225
Max Cap
Maximum Flow: 564
Number of Augmenting Paths: 28
Average Length of Augmenting Paths (ML): 4.214285714285714
Mean Proportional Length (MPL): 1.0535714285714286
Total Number of Edges: 1225
Random
Maximum Flow: 564
Number of Augmenting Paths: 422
Average Length of Augmenting Paths (ML): 34.95023696682465
Mean Proportional Length (MPL): 8.737559241706162
Total Number of Edges: 1225
